# Modifying E.coli Carbon Byproduct Secretion Profile

This notebook introduce the necessary steps through weighting flux when using parsimonious FBA(pFBA) as optimization method. This approach restricts production of acetate as sole carbon byproduct from E. coli, 

In [1]:
%cd ..

d:\scarccpy


In [23]:
import cobra
import pandas as pd
from cobra.flux_analysis.parsimonious import pfba

E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")

In [24]:
E0.medium = {
    'EX_lcts_e': 10,
    'EX_met__L_e': .02,
    'EX_pi_e': 10,
    'EX_fe3_e': 10,
    'EX_mn2_e': 10,
    'EX_fe2_e': 10,
    'EX_zn2_e': 10,
    'EX_mg2_e': 10,
    'EX_ca2_e': 10,
    'EX_ni2_e': 10,
    'EX_cu2_e': 10,
    'EX_cobalt2_e': 10,
    'EX_mobd_e': 10,
    'EX_so4_e': 10,
    'EX_k_e': 10,
    'EX_cl_e': 10,
    'EX_o2_e': 10,
    'EX_nh4_e': 10}

Case without optimize for total flux 

In [25]:
E_col = ['EX_lcts_e', 'EX_gal_e','EX_co2_e','EX_ac_e', 'EX_met__L_e','BIOMASS_Ec_iML1515_core_75p37M']
fba_sol = E0.optimize().to_frame().T
fba_sol.loc['fluxes',E_col]


EX_lcts_e                        -4.874831
EX_gal_e                          0.000000
EX_co2_e                          0.112072
EX_ac_e                           0.000000
EX_met__L_e                      -0.020000
BIOMASS_Ec_iML1515_core_75p37M    0.129945
Name: fluxes, dtype: float64

Parsimonious FBA

In [26]:
psol = pfba(E0).to_frame().T
print(psol.loc['fluxes', E_col])

EX_lcts_e                        -1.214347
EX_gal_e                          0.711305
EX_co2_e                          4.959373
EX_ac_e                           0.055652
EX_met__L_e                      -0.020000
BIOMASS_Ec_iML1515_core_75p37M    0.129945
Name: fluxes, dtype: float64


Where is lactose broken down?

In [6]:
print(str(E0.reactions.get_by_id('LACZpp')), psol.loc['fluxes', 'LACZpp'])
print(str(E0.reactions.get_by_id('LACZ')), psol.loc['fluxes', 'LACZpp'])
print(str(E0.reactions.get_by_id('LCTStpp')), psol.loc['fluxes', 'LACZpp'])

LACZpp: h2o_p + lcts_p --> gal_p + glc__D_p 0.7113047581497066
LACZ: h2o_c + lcts_c --> gal_c + glc__D_c 0.7113047581497066
LCTStpp: h_p + lcts_p --> h_c + lcts_c 0.7113047581497066


Galactose is secreted because half of lactose is break down in periplasm compartment, as soon as galactose is produced, it become readily to be secreted to extracellular space. Which is undesirable as bacteria rarely secret complex carbon as byproduct.

<div class="alert alert-info">

**NOTE :** Default FBA solution balances reduced cost for the flux, leading to flux equal quantity flowing through the two availble pathways
</div>

## Flux weighting

We can impose extra cost in the flux secretion for undesirable galactose secretion. 

For weight = 10,

\begin{align}
Unweighted: gal_p \rightarrow gal_e \\
Weighted: 0.1 \times gal_p \rightarrow 0.1 \times gal_e
\end{align}

- Increasing the cost associated with galactose secretion flux within the parsimonious optimization procedure decreases the likelihood of that flux being utilized.
    - To transport 1 unit of gal to extracellular space, Eq. (1) only needs 1 unit of flux
    - To transport 1 unit of gal to extracellular space, Eq. (2) needs 10 unit of flux
- Weighting the secreting flux in opposite direction for acetate reduce the cost associated with acetate secretion
    - fermentation could be more favorable than respiration 
    - limited supply of methionine in co-culture imply selecting pathway with lower proteomic cost 

<div class="alert alert-info">

**NOTE:** 
This implementation do not alter functionality of metabolic network. ONLY flux quantity evaluated in pFBA is changed.

</div>


**COMETS implementation:** 

Directly changing stoichiometry for exchange reaction is not properly implemented in COMETS. The following implementation is adopted instead

For weight w = 10,

\begin{align}
Precursor \quad reactions \quad producing \quad gal_p &: A/10 +B/10 \rightarrow frac\_gal_p +B/10 \\
Weighted \quad secretion \quad flux &: frac\_gal_p \rightarrow frac\_gal_e \\ 
\end{align}

\begin{align*}
\text{where } frac\_gal_p = 0.1 \times gal_p \text{ and }  frac\_gal_e = 0.1 \times gal_e 
\end{align*}

Vice versa for w=0.1 $\Rightarrow$ $bulk\_ac$ = $10 \times ac$. Frac and bulk are fraction and bulk count of original compound


Execution:  

class ``BasicModel`` reads model files, allows for flux weighting and renaming metabolites name to ensure the exchange of metabolites are of the same kind in COMETS. Toggle flux_weighting to switch. Get the models by calling ``load_ES_models()``

In [60]:
import os
from scarcc.preparation.find_directory import find_directory
from scarcc.preparation.metabolic_model import BasicModel

model_directory = find_directory('models', os.path.abspath(''))
E0, S0, all_components = BasicModel(flux_weighting=True, 
                                    ac_scale=10, # Optional
                                    gal_scale=1/3, # Optional
                                    model_directory=model_directory).load_ES_models()

gal_scale -0.6666666666666667


In [8]:
str(E0.reactions.ACt2rpp), str(E0.reactions.EX_bulk_ac_e)

('ACt2rpp: 0.9999999999999998 bulk_ac_p + 10.0 h_p <=> 10.0 ac_c + 10.0 h_c',
 'EX_bulk_ac_e: bulk_ac_e --> ')

## Secretion profiles after flux weighting

In [32]:
E_col_w = ['EX_lcts_e', 'EX_gal_e','EX_co2_e','EX_bulk_ac_e', 'EX_met__L_e','BIOMASS_Ec_iML1515_core_75p37M']
fba_sol_w = E0.optimize().to_frame().T
psol_w =pfba(E0).to_frame().T
print(psol_w.loc['fluxes', E_col_w])

EX_lcts_e                        -2.612578
EX_gal_e                          0.000000
EX_co2_e                          0.000000
EX_ac_e                           0.055652
EX_met__L_e                      -0.020000
BIOMASS_Ec_iML1515_core_75p37M    0.129945
Name: fluxes, dtype: float64

In [59]:
compare_pfba = pd.concat([psol.loc[['fluxes'], E_col], psol_w.loc[['fluxes'], E_col_w]])
compare_pfba.index = ['pfba without flux weighting', 'pfba with flux weighting']
compare_pfba

,EX_lcts_e,EX_gal_e,EX_co2_e,EX_ac_e,EX_met__L_e,BIOMASS_Ec_iML1515_core_75p37M,EX_bulk_ac_e
pfba without flux weighting,-1.214347,0.711305,4.959373,0.055652,-0.02,0.129945,NaN
pfba with flux weighting,-1.208707,0.000000,4.022391,NaN,-0.02,0.129945,0.261813


Acetate quantity after implementing flux weighting increase from 0.0557 to 2.6 molecules. Galactose secretion flux vanished